# Queues

Queues use **FIFO** or **firs in, first out** ordering, meaning the first element that was added will always be the first one removed. Queues are handy when you need to maintain the order of your elements to process later.

## Common Operations

In [9]:
interface Queue<T: Any> {
    fun enqueue(element: T): Boolean
    fun dequeue(): T?
    val count: Int
        get
    val isEmpty: Boolean
        get() = count == 0
    fun peek(): T?
}

- `enqueue`: Inserts an element at the back of the queue and returns `true`if the operation is successful.
- `dequeue`: Removes the element at the front of the queue and returns it.
- `isEmpty`: Checks if the queue is empty using the `count` property.
- `peek`: Returns the element at the front of the queue without removing it.

![Queue](queue.png)


## List-based implementation

In [10]:
class ArrayListQueue<T: Any> : Queue<T> {
    private val list = arrayListOf<T>()

    override val count: Int
        get() = list.size

    override fun peek(): T? = list.getOrNull(0)

    override fun enqueue(element: T): Boolean {
        list.add(element)
        return true
    }

    override fun dequeue(): T? = 
        if (isEmpty) null else list.removeAt(0)

    override fun toString(): String = list.toString()
}

`enqueue` is an **O(1)** operation. This is because the list has empty space at the back.

![Queue](queue.png)

After adding multiple element, the internal array of the ArrayList will eventually be full. When you want to use more than the allocated space, the array must resize to make additional room.

![Resizing](queue-resizing.png)

Resizing is an **O(n)** operation. Since this doesn't happen very often, the complexity still works out to be an amortized O(1).

`dequeue`is an **O(n)** operation. You remove the element from the beginning of the list, it requires all the remaining elements in the list to be shifted in memory.

In [11]:
println("Queue with ArrayList")
val queue = ArrayListQueue<String>().apply { 
    enqueue("Ray")
    enqueue("Brian")
    enqueue("Eric")
 }
 println(queue)
 val dequeued = queue.dequeue()
 println("Dequeued: $dequeued")
 println(queue)

Queue with ArrayList
[Ray, Brian, Eric]
Dequeued: Ray
[Brian, Eric]


![Array-Based Queue](queue-arraylist-metrics.png)

## Doubly linked list implementation

In [12]:
data class Node<T : Any>(
    var value: T,
    var next: Node<T>? = null,
    var previous: Node<T>? = null
) {
    override fun toString(): String {
        return if (next != null) {
            "$value -> ${next.toString()}"
        } else {
            "$value"
        }
    }
}

In [13]:
class DoublyLinkedList<T : Any> {

    fun isEmpty(): Boolean {
        return head == null
    }

    private var head: Node<T>? = null
    private var tail: Node<T>? = null


    override fun toString(): String {
        if (isEmpty()) {
            return "Empty list"
        }
        return head.toString()
    }

    fun append(value: T) {
        val newNode = Node(value = value, previous = tail)
        if (isEmpty()) {
            head = newNode
            tail = newNode
            return
        }
        tail?.next = newNode

        tail = newNode
    }

    fun node(index: Int): Node<T>? {
        var currentNode = head
        var currentIndex = 0

        while (currentNode != null && currentIndex < index) {
            currentNode = currentNode.next
            currentIndex += 1
        }
        return currentNode
    }

    fun remove(node: Node<T>): T {
        val prev = node.previous
        val next = node.next

        if (prev != null) {
            prev.next = next
        } else {
            head = next
        }
        next?.previous = prev
        if (next == null) {
            tail = prev
        }
        node.previous = null
        node.next = null
        return node.value
    }

    val first: Node<T>?
        get() = head

}

In [16]:


class LinkedLisQueue<T: Any> : Queue<T> {
    private val list = DoublyLinkedList<T>()
    
    private var size = 0

    override val count: Int
        get() = size

    // 1
    override fun enqueue(element: T): Boolean {
        list.append(element)
        size++
        return true
    }

    // 2
    override fun dequeue(): T? {
        val firstNode = list.first ?: return null
        size--
        return list.remove(firstNode)
    }

    override fun peek(): T? = list.first?.value

    override fun toString(): String = list.toString()
}

1. Behind the scenes, the doubly linked list will update its tail node's previous and next references to the new node. This ia an **O(1)** operation.
![Enqueue](enqueue-doublell.png)
2. Removing from the front of the list is also an **O(1)** operation. The doubly linked list updates the `next` abd `previous` pointer between the first two nodes of the linked list.
![Dequeue](dequeue-doublell.png)  

In [17]:
println("Queue with Double Linked List")
val queue = LinkedLisQueue<String>().apply { 
    enqueue("Ray")
    enqueue("Brian")
    enqueue("Eric")
 }
 println(queue)
 val dequeued = queue.dequeue()
 println("Dequeued: $dequeued")
 println(queue)
 println("Next up: ${queue.peek()}")

Queue with Double Linked List
Ray -> Brian -> Eric
Dequeued: Ray
Brian -> Eric
Next up: Brian


![Linked-List Based Queue](queue-doublell-metrics.png)

The main weaknesses with `LinkedListQueue` is that each element has to have extra storage for the forward and back reference and every time you create a new element, it requires relatively expensive dynamic allocation.

# Ring buffer implementation

In [19]:
class RingBuffer<T : Any>(private val size: Int) {

  private var array = ArrayList<T?>(size)
  private var readIndex = 0
  private var writeIndex = 0

  val count: Int
    get() = availableSpaceForReading

  private val availableSpaceForReading: Int
    get() = (writeIndex - readIndex)

  val first: T?
    get() = array.getOrNull(readIndex)

  val isEmpty: Boolean
    get() = (count == 0)

  private val availableSpaceForWriting: Int
    get() = (size - availableSpaceForReading)

  val isFull: Boolean
    get() = (availableSpaceForWriting == 0)

  fun write(element: T): Boolean {
    return if (!isFull) {
      if (array.size < size) {
        array.add(element)
      } else {
        array[writeIndex % size] = element
      }
      writeIndex += 1
      true
    } else {
      false
    }
  }

  fun read(): T? {
    return if (!isEmpty) {
      val element = array[readIndex % size]
      readIndex += 1
      element
    } else {
      null
    }
  }

  override fun toString(): String {
    val values = (0 until availableSpaceForReading).map { offset ->
      "${array[(readIndex + offset) % size]!!}"
    }
    return values.joinToString(prefix = "[", separator = ", ", postfix = "]")
  }

}

In [20]:
class RingBufferQueue<T: Any>(size: Int): Queue<T> {
    private val ringBuffer = RingBuffer<T>(size)

    override val count: Int
        get() = ringBuffer.count

    override fun peek(): T? = ringBuffer.first

    override fun enqueue(element: T): Boolean = ringBuffer.write(element)

    override fun dequeue(): T? = 
        if (isEmpty) null else ringBuffer.read()

    override fun toString(): String = ringBuffer.toString()
}

In [21]:
println("Queue with Ring Buffer")
val queue = RingBufferQueue<String>(10).apply { 
    enqueue("Ray")
    enqueue("Brian")
    enqueue("Eric")
 }
 println(queue)
 println("Dequed: ${queue.dequeue()}")
 println(queue)
 println("Next up: ${queue.peek()}")

Queue with Ring Buffer
[Ray, Brian, Eric]
Dequed: Ray
[Brian, Eric]
Next up: Brian


![Ring-Buffer Based Queue](queue-ring-metrics.png)

The ring buffer has a fixed size, which means that enqueue can fail.

# Double-stack implementation

In [22]:
interface Stack<T : Any> {
    fun push(element: T)

    fun pop(): T?

    fun peek(): T?

    val count: Int
        get

    val isEmpty: Boolean
        get() = count == 0
}

class StackImpl<T : Any> : Stack<T> {
    private val storage = arrayListOf<T>()

    override fun toString() = buildString {
        appendLine("----top----")
        storage.asReversed().forEach {
            appendLine("$it")
        }
        appendLine("-----------")
    }

    override val count: Int
        get() = storage.size

    override fun push(element: T) {
        storage.add(element)
    }

    override fun pop(): T? {
        if (isEmpty) {
            return null
        }
        return storage.removeAt(count - 1)
    }

    override fun peek(): T? {
        return storage.lastOrNull()
    }

    companion object {
        fun <T : Any> create(items: Iterable<T>): Stack<T> {
            val stack = StackImpl<T>()
            for (item in items) {
                stack.push(item)
            }
            return stack
        }
    }
}

fun <T : Any> stackOf(vararg elements: T): Stack<T> {
    return StackImpl.create(elements.asList())
}

Whenever you enqueue an element, it goes in the right stack. When you need to dequeue an element, you reverse the right stack and place it in the **left** stack so that you can retrieve the elements using FIFO order.
![Queue with Double Stack](queue_double_stack.png)

In [26]:
class StackQueue<T: Any> : Queue<T> {
    private val leftStack = StackImpl<T>()
    private val rightStack = StackImpl<T>()

    override val count: Int
        get() = leftStack.count + rightStack.count

    override val isEmpty: Boolean
        get() = leftStack.isEmpty && rightStack.isEmpty

    override fun peek(): T? {
        if (leftStack.isEmpty) transferElements()
        return leftStack.peek()
    }

    override fun enqueue(element: T): Boolean {
        rightStack.push(element)
        return true
    }

    override fun dequeue(): T? {
        if (leftStack.isEmpty) transferElements()
        return leftStack.pop()
    }
        
    // you pop elements from the right stack and push them into the left stack.
    private fun transferElements() {
        var nextElement = rightStack.pop()
        while (nextElement != null) {
            leftStack.push(nextElement)
            nextElement = rightStack.pop()
        }
    }

    override fun toString(): String {
        return "Left stack \n$leftStack Righ stack: \n$rightStack"
    }
}

If you have a lot of elements in the right stack, calling `peek()` will be **O(n)** for just that one call when it has to move all of those elements. Any further call will be **O(1)**.
Remember, you only transfer the elements in the right stack when the left stack is empty. This makes `dequeue()` an amortized **O(1)** operation, just like `peek()`. 

In [27]:
println("Queue with Double Stack")
val queue = StackQueue<String>().apply { 
    enqueue("Ray")
    enqueue("Brian")
    enqueue("Eric")
 }
 println(queue)
 println("Dequed: ${queue.dequeue()}")
 println(queue)
 println("Next up: ${queue.peek()}")

Queue with Double Stack
Left stack 
----top----
-----------
 Righ stack: 
----top----
Eric
Brian
Ray
-----------

Dequed: Ray
Left stack 
----top----
Brian
Eric
-----------
 Righ stack: 
----top----
-----------

Next up: Brian


# Challenge 3: Monopoly

Create a Monopoly organizer that tells you whose turn it is.

In [35]:
fun <T : Any> Queue<T>.nextPlayer(): T? {
    val next = dequeue()
    if (next != null) enqueue(next)
    return next
}

val players = StackQueue<String>().apply { 
    enqueue("Jose")
    enqueue("Peter")
    enqueue("Vicent")
    enqueue("Parker")
 }
 for (i in 0..10) {
     println("NextPlayer: ${players.nextPlayer()}")
 }

NextPlayer: Jose
NextPlayer: Peter
NextPlayer: Vicent
NextPlayer: Parker
NextPlayer: Jose
NextPlayer: Peter
NextPlayer: Vicent
NextPlayer: Parker
NextPlayer: Jose
NextPlayer: Peter
NextPlayer: Vicent


# Challenge 4: Reverse data

Implement a method to reverse the contents of a queue using an extension function.

In [39]:
fun <T: Any> Queue<T>.reverse() {
    val stack = StackImpl<T>()
    while (peek() != null) {
        stack.push(dequeue()!!)
    }
    while (stack.peek() != null) {
        enqueue(stack.pop()!!)
    }
}

val players = LinkedLisQueue<String>().apply {
    enqueue("Jose")
    enqueue("Peter")
    enqueue("Vicent")
    enqueue("Parker")
}
println("Original List:")
println(players)
println("Reversed:")
players.reverse()
println(players)

Original List:
Jose -> Peter -> Vicent -> Parker
Reversed:
Parker -> Vicent -> Peter -> Jose
